In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pickle
from bs4 import BeautifulSoup
import time

### URL 이용 프레임 소스 다운로드

In [ ]:
chrome_driver = '/Users/woon/Desktop/chromedriver'
driver = webdriver.Chrome(chrome_driver)
url = 'http://kipris.or.kr/kdtj/remocon/frame.jsp?kind=2&start=DR&userid=&menuKind=9&rtField&rights=TM'

driver.implicitly_wait(5)
driver.get(url)

driver.switch_to.frame(driver.find_element_by_name("right"))
src = driver.page_source
soup = BeautifulSoup(src)

driver.close()
soup

### 대분류 딕트 pickle 파일 생성

In [ ]:
l_category = []
for i in range(len(soup.select('a'))):
    l_category.append(soup.select('a')[i].get_text())


In [ ]:
del l_category[0:2]
del l_category[-1]
del l_category[-1]

In [ ]:
l_category
large_number = []
large_text = []
for i in range(len(l_category)):
    if(i%2==1):
        large_text.append(l_category[i])
    else:
        large_number.append(l_category[i])

In [ ]:
large_zip = zip(large_number,large_text)
main_category = dict(large_zip)

main_category

### 중분류 딕트 pickle 파일 생성

In [ ]:
medium_value = []
for i in range(len(soup.find_all('td', colspan='2'))):
    medium_value.append(soup.find_all('td', colspan='2')[i].get_text())

In [ ]:
len(medium_value)

In [ ]:
m_number = []
for i in range(len(soup.select('td.line'))):
    if soup.select('td.line')[i].get_text() !='':
        m_number.append(soup.select('td.line')[i].get_text())

In [ ]:
m_number

In [ ]:
medium = []
l_number = 1
for i in range(len(m_number)):
    if i == 0:
        medium.append('0101')
    else:
        if int(m_number[i]) < int(m_number[i-1]):
            l_number += 1
            medium.append(str(l_number)+m_number[i])
        else:
            medium.append(str(l_number)+m_number[i])

In [ ]:
for i in range(len(medium)):
    if len(medium[i]) == 3:
        medium[i] = '0'+medium[i]

In [ ]:
len(medium),len(medium_value)

In [ ]:
m_zip = zip(medium,medium_value)
medium_dict = dict(m_zip)
medium_dict

### 소분류 딕트 pickle 파일 생성

In [ ]:
soup.select('td.txt_left')

In [ ]:
vnumber = []
for i in range(len(soup.select('input')[7:])):
    vnumber.append(soup.select('input')[i+7]['value'])
del vnumber[-1] #마지막 값에 nan 들어있음

In [ ]:
vtext = []
for i in range(len(soup.select('td.txt_left'))):
    vtext.append(soup.select('td.txt_left')[i].get_text())

In [ ]:
len(vnumber),len(vtext)

In [ ]:
myzip = zip(vnumber,vtext)
vienna_dict = dict(myzip)
vienna_dict

# 대분류,중분류,소분류 Dict 통합 파일 생성

In [ ]:
main_category.update(medium_dict)
main_category.update(vienna_dict)
len(main_category)

In [ ]:
with open('vienna_dict.pickle','wb') as f:
    pickle.dump(main_category, f, pickle.HIGHEST_PROTOCOL)